In [ ]:
pip install imbalanced-learn

In [ ]:
pip install sentence-transformers

In [ ]:
pip install keras-tuner

In [ ]:
pip install seaborn

In [ ]:
pip install flask_cors

In [ ]:
pip install tensorflow-hub

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install numpy

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install tensorflow keras-tuner numpy matplotlib

In [ ]:
pip show tensorflow keras scikit-learn

In [2]:
#Moods code
import json
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))

# Load short mood dataset
with open("short_mood_dataset.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

# Normalize and group contexts by mood
mood_to_contexts = defaultdict(list)
for sample in data:
    mood = sample.get("Mood", "").strip().title()
    context = sample.get("Context", "").strip().lower()
    if mood and context:
        mood_to_contexts[mood].append(context)

# Extract keywords per mood
mood_keywords = {}
for mood, contexts in mood_to_contexts.items():
    if len(contexts) < 2:
        continue  # skip rare moods
    all_text = " ".join(contexts)
    tokens = word_tokenize(all_text)
    cleaned = [w for w in tokens if w.isalpha() and w.lower() not in stop_words]
    top_keywords = [word.lower() for word, _ in Counter(cleaned).most_common(15)]
    if top_keywords:
        mood_keywords[mood] = top_keywords

# Save result
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(mood_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json generated with {len(mood_keywords)} moods.")


# Load the old mood_keywords.json (with Mood_0, Mood_1... keys)
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

# Define mapping from generic Mood_# labels to real mood names
mood_map = {
    "Mood_0": "Abuse",
    "Mood_1": "Sexual Issues",
    "Mood_2": "Erectile Dysfunction",
    "Mood_3": "Relationship Issues",
    "Mood_4": "Depression",
    "Mood_5": "Child Custody",
    "Mood_6": "Parental Issues",
    "Mood_7": "Infidelity",
    "Mood_8": "Marriage Problems",
    "Mood_9": "Motherhood",
    "Mood_10": "Gender Identity",
    "Mood_11": "Therapist Issues",
    "Mood_12": "Therapy Anxiety",
    "Mood_13": "Marital Issues",
    "Mood_14": "Unrequited Love",
    "Mood_15": "Therapy Nervousness",
    "Mood_16": "Health Trauma",
    "Mood_17": "PTSD",
    "Mood_18": "Alcoholism",
    "Mood_19": "Breakups",
    "Mood_20": "Sexless Marriage",
    "Mood_21": "Hearing Voices",
    "Mood_22": "Eating Disorders",
    "Mood_23": "Suicidal Thoughts",
    "Mood_24": "Panic Attacks",
    "Mood_25": "Toxic Relationships",
    "Mood_26": "Anxiety And Depression",
    "Mood_27": "Daughter Stress",
    "Mood_28": "Work And Family Stress",
    "Mood_29": "School Stress"
}

# Create a new dict with renamed keys
renamed_keywords = {}
for mood_id, keywords in mood_keywords.items():
    readable_label = mood_map.get(mood_id, mood_id)  # fallback to original key if no match
    renamed_keywords[readable_label] = keywords

# Save the updated file
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(renamed_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json updated and saved with {len(renamed_keywords)} mood labels.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✅ mood_keywords.json generated with 30 moods.
✅ mood_keywords.json updated and saved with 30 mood labels.


In [3]:
import json
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download necessary NLTK data (run this once)
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    word_tokenize("example")
except LookupError:
    nltk.download('punkt')

# ---- Step 1: Load original mood_keywords.json ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

# ---- Step 2: Map Mood_# to actual mood names ----
mood_map = {
    "Mood_0": "Abuse",
    "Mood_1": "Sexual Issues",
    "Mood_2": "Erectile Dysfunction",
    "Mood_3": "Relationship Issues",
    "Mood_4": "Depression",
    "Mood_5": "Child Custody",
    "Mood_6": "Parental Issues",
    "Mood_7": "Infidelity",
    "Mood_8": "Marriage Problems",
    "Mood_9": "Motherhood",
    "Mood_10": "Gender Identity",
    "Mood_11": "Therapist Issues",
    "Mood_12": "Therapy Anxiety",
    "Mood_13": "Marital Issues",
    "Mood_14": "Unrequited Love",
    "Mood_15": "Therapy Nervousness",
    "Mood_16": "Health Trauma",
    "Mood_17": "PTSD",
    "Mood_18": "Alcoholism",
    "Mood_19": "Breakups",
    "Mood_20": "Sexless Marriage",
    "Mood_21": "Hearing Voices",
    "Mood_22": "Eating Disorders",
    "Mood_23": "Suicidal Thoughts",
    "Mood_24": "Panic Attacks",
    "Mood_25": "Toxic Relationships",
    "Mood_26": "Anxiety And Depression",
    "Mood_27": "Daughter Stress",
    "Mood_28": "Work And Family Stress",
    "Mood_29": "School Stress"
}

# Rename original mood keys
renamed_keywords = {}
for mood_id, keywords in mood_keywords.items():
    readable_label = mood_map.get(mood_id, mood_id)
    renamed_keywords[readable_label] = keywords

# ---- Step 3: Define the 4 main categories and map moods under them ----
category_map = {
    "Depression & Anxiety": [
        "Depression", "Unrequited Love", "Breakups", "Suicidal Thoughts",
        "Anxiety And Depression", "Therapy Anxiety", "Therapy Nervousness",
        "Panic Attacks", "Hearing Voices"
    ],
    "Personality & Behaviour": [
        "Gender Identity", "Motherhood", "Alcoholism", "Eating Disorders"
    ],
    "Stress & Coping": [
        "Work And Family Stress", "Daughter Stress", "School Stress",
        "Toxic Relationships", "Parental Issues", "Child Custody",
        "Relationship Issues", "Marriage Problems", "Infidelity",
        "Marital Issues", "Therapist Issues"
    ],
    "Trauma & PTSD": [
        "Abuse", "Sexual Issues", "Erectile Dysfunction", "Health Trauma",
        "PTSD", "Sexless Marriage"
    ]
}

# ---- Step 4: Function to generate additional keywords using NLTK ----
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

def generate_keywords(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        w for w in tokens if not w in stop_words and not w in punctuation and w.isalpha()
    ]
    return filtered_tokens

# Example of adding more descriptive text for some moods to extract keywords
mood_descriptions = {
    "Depression": "Feeling sad, loss of interest, fatigue, changes in appetite and sleep, feelings of worthlessness, difficulty concentrating, thoughts of death or suicide. Major depressive disorder symptoms.",
    "Anxiety And Depression": "Experiencing both persistent worry, nervousness, and fear along with feelings of sadness, hopelessness, and loss of interest in activities.",
    "Relationship Issues": "Problems in interpersonal connections, including communication breakdowns, conflicts, lack of trust, and emotional distance between partners, family members, or friends.",
    "PTSD": "A condition that develops after a traumatic event, characterized by flashbacks, nightmares, severe anxiety, and uncontrollable thoughts about the event. Post-traumatic stress disorder symptoms.",
    "Abuse": "Experiencing physical, emotional, sexual, or financial harm from another person, leading to trauma and significant distress.",
    "Gender Identity": "A person's internal sense of being male, female, both, neither, or somewhere else along the gender spectrum.",
    "Work And Family Stress": "Strain and pressure arising from the demands and responsibilities of both professional life and family obligations.",
    "Eating Disorders": "Characterized by persistent disturbances of eating or eating-related behavior that results in the altered consumption or absorption of food and that significantly impairs physical health or psychosocial functioning."
}

# ---- Step 5: Group and merge keywords for main categories, including generated ones ----
grouped_keywords = {}
additional_keywords_count = 5  # Number of additional keywords to add per mood

for main_category, specific_moods in category_map.items():
    combined_keywords = []
    for mood in specific_moods:
        # Add original keywords
        combined_keywords.extend(renamed_keywords.get(mood, []))

        # Generate and add more keywords based on descriptions if available
        if mood in mood_descriptions:
            description = mood_descriptions[mood]
            generated_keywords = generate_keywords(description)
            combined_keywords.extend(generated_keywords)

    # Count and deduplicate, keeping top 20
    counter = Counter(combined_keywords)
    grouped_keywords[main_category] = [kw for kw, _ in counter.most_common(20)]

# ---- Step 6: Save the final grouped mood keywords ----
with open("mood_keywords_grouped_enhanced.json", "w", encoding="utf-8") as f:
    json.dump(grouped_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords_grouped_enhanced.json created with {len(grouped_keywords)} main categories, including additional keywords.")

✅ mood_keywords_grouped_enhanced.json created with 4 main categories, including additional keywords.


In [4]:
import os
import json
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model,save_model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Layer, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]
contexts = [sample["Context"] for sample in parsed_data]

# ---- Load Mood Keywords ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

def detect_mood(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

X_texts, Y_labels = [], []
for context in contexts:
    mood = detect_mood(context)
    if mood:
        X_texts.append(context)
        Y_labels.append(mood)

# ---- Encode and Filter Labels ----
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_labels)
with open("mood_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

label_counts = Counter(Y_encoded)
valid_labels = {label for label, count in label_counts.items() if count > 1}
filtered = [(x, y) for x, y in zip(X_texts, Y_encoded) if y in valid_labels]
X_texts_filtered = [x for x, _ in filtered]
Y_filtered = [y for _, y in filtered]

# ---- SBERT Embedding with Save/Load Logic ----
embedding_file = "sbert_embeddings.npy"
texts_file = "sbert_embedding_texts.pkl"
labels_file = "sbert_embedding_labels.pkl"

# Check if embeddings already exist
if os.path.exists(embedding_file) and os.path.exists(texts_file) and os.path.exists(labels_file):
    print("📂 Loading existing SBERT embeddings...")
    X_emb = np.load(embedding_file)

    with open(texts_file, "rb") as f:
        saved_texts = pickle.load(f)

    with open(labels_file, "rb") as f:
        saved_labels = pickle.load(f)

    # Verify that the saved data matches current data
    if len(saved_texts) == len(X_texts_filtered) and len(saved_labels) == len(Y_filtered):
        print("✅ Loaded existing embeddings successfully!")
    else:
        print("⚠️ Saved data doesn't match current data. Regenerating embeddings...")
        sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
        print("🔄 Generating SBERT embeddings...")
        X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)

        # Save the embeddings and corresponding data
        print("💾 Saving SBERT embeddings...")
        np.save(embedding_file, X_emb)
        with open(texts_file, "wb") as f:
            pickle.dump(X_texts_filtered, f)
        with open(labels_file, "wb") as f:
            pickle.dump(Y_filtered, f)
else:
    # Generate embeddings if they don't exist
    sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
    print("🔄 Generating SBERT embeddings...")
    X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)

    # Save the embeddings and corresponding data
    print("💾 Saving SBERT embeddings...")
    np.save(embedding_file, X_emb)
    with open(texts_file, "wb") as f:
        pickle.dump(X_texts_filtered, f)
    with open(labels_file, "wb") as f:
        pickle.dump(Y_filtered, f)

# ---- Resampling ----
min_samples = min(Counter(Y_filtered).values())
if min_samples < 2:
    print("⚠️ SMOTE skipped due to insufficient samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered
else:
    smote_k = max(1, min(5, min_samples - 1))
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_emb, Y_filtered)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote)

Y_resampled = to_categorical(Y_resampled_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.25, stratify=np.argmax(Y_resampled, axis=1), random_state=42
)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

class_weights = compute_class_weight("balanced", classes=np.unique(Y_train_int), y=Y_train_int)
class_weight_dict = dict(enumerate(class_weights))

# ---- Model Builder ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout=0.3):
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention = Dropout(dropout)(attention)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention]))
        ff = Dense(ff_dim, activation='relu')(out1)
        ff = Dense(inputs.shape[-1])(ff)
        ff = Dropout(dropout)(ff)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- Train Model ----
gc.collect()
model = build_model(input_shape=X_train.shape[1:], num_classes=Y_resampled.shape[1])
early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, min_lr=1e-6, verbose=1)


print("Length of X_train:", len(X_train))
print("Length of X_test:", len(X_test))
print("Length of Y_train_int:", len(Y_train_int))
print("Length of Y_test_int:", len(Y_test_int))

e:\AI ML Models\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Loading existing SBERT embeddings...
✅ Loaded existing embeddings successfully!


e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator.


Length of X_train: 34908
Length of X_test: 11637
Length of Y_train_int: 34908
Length of Y_test_int: 11637


In [ ]:
print("🚀 Training model...")
history = model.fit(
    X_train, Y_train_int,
    validation_data=(X_test, Y_test_int),
    epochs=20,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

# ---- Plot Results ----
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.title('Accuracy'); plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Loss'); plt.legend()
plt.tight_layout()
plt.show()

# ---- F1 Score ----
Y_pred_probs = model.predict(X_test, verbose=1)
Y_pred = np.argmax(Y_pred_probs, axis=1)
f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")


model_path = "mood_classifier_model.keras"

# 🧹 Remove old `.keras` model folder if it exists
if os.path.exists(model_path):
    try:
        if os.path.isfile(model_path):
            os.remove(model_path)
        else:
            import shutil
            shutil.rmtree(model_path)
        print(f"🧹 Removed old model: {model_path}")
    except Exception as e:
        print(f"❌ Could not delete old model: {e}")
        raise

# 💾 Save the model using native format
try:
    save_model(model, model_path)
    print(f"✅ Model saved in native Keras format at: {model_path}")
except Exception as e:
    print(f"❌ Error saving model: {e}")

🚀 Training model...
Epoch 1/20
  24/1091 ━━━━━━━━━━━━━━━━━━━━ 1:59 112ms/step - accuracy: 0.9900 - loss: 0.0471

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# ---- Confusion Matrix ----f
cm = confusion_matrix(Y_test_int, Y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("🧩 Confusion Matrix")
plt.tight_layout()
plt.show()

# ---- Optional: ROC AUC for binary/multi-class (One-hot) ----
try:
    from sklearn.preprocessing import label_binarize
    Y_test_bin = label_binarize(Y_test_int, classes=list(range(len(label_encoder.classes_))))
    roc_auc = roc_auc_score(Y_test_bin, Y_pred_probs, average="macro", multi_class="ovr")
    print(f"🚀 ROC AUC Score (macro): {roc_auc:.4f}")
except Exception as e:
    print(f"⚠️ Skipped ROC AUC Score: {e}")

In [ ]:
import os
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# --- Custom Attention Layer ---
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# --- Load Model and Assets ---
model = load_model("mood_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("mood_keywords_grouped_enhanced.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

# --- Parse cleaned_questions.json ---
with open("cleaned_questions.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Structure: {category.lower(): [questions...]}
question_data = {}
for entry in raw_data:
    category = entry.get("category", "").strip().lower()
    if not category:
        continue
    all_qs = []
    for doc in entry.get("questions", []):
        all_qs.extend([q.strip() for q in doc.get("questions", []) if isinstance(q, str) and len(q.strip()) > 10])
    if all_qs:
        question_data[category] = all_qs

# --- Logging ---
def log_entry(data):
    path = "chat_log.json"
    logs = []
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            logs = json.load(f)
    logs.append(data)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)

# --- Mood Detection ---
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(k in text for k in keywords):
            return mood
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text)
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood, 1.0

    emb = sbert.encode([text])
    emb = np.expand_dims(emb, axis=1)
    pred = model.predict(emb, verbose=0)
    idx = np.argmax(pred)
    conf = float(np.max(pred))
    mood = label_encoder.inverse_transform([idx])[0]
    print(f"🤖 Predicted mood: {mood} (confidence: {conf:.2f})")
    return mood, conf

# --- Question Fetch ---
def fetch_questions(mood, count=3):
    key = mood.strip().lower()
    questions = question_data.get(key, [])
    if not questions:
        print(f"⚠️ No questions found for mood '{mood}'.")
        return []
    return list(np.random.choice(questions, size=min(count, len(questions)), replace=False))

# --- Chatbot ---
def chatbot():
    print("🧠 MoodBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood, confidence = predict_mood(user_input)
        questions = fetch_questions(mood)

        if not questions:
            print(f"🧠 MoodBot: I'm here to support you. Would you like to talk more about feeling {mood.lower()}?\n")
            continue

        print(f"\n🧠 MoodBot ({mood.title()}): It seems you're feeling {mood.lower()}. Let's explore further:\n")
        for i, q in enumerate(questions, 1):
            answer = input(f"{i}. {q}\nYou: ").strip()
            log_entry({
                "timestamp": datetime.now().isoformat(),
                "user_input": user_input,
                "predicted_mood": mood,
                "confidence": confidence,
                "question": q,
                "answer": answer
            })
        print("\n✅ Thank you for sharing. I'm here anytime you want to talk.\n")

# --- Run Chatbot ---
if __name__ == "__main__":
    chatbot()